# Run a classifer on the branchwater search output

In [1]:
# base checkout of workflow directory is here:
BASE='/home/ctbrown/scratch3/2025-workflow-core99/'

# parquet files from branchwater
BASE_OUTPUTS=BASE+'/outputs.branchwater'

In [2]:
import polars as pl
import numpy as np
import sklearn.tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

In [3]:
dirpath = BASE_OUTPUTS+'/*.parquet'

bw_df = pl.scan_parquet(dirpath).collect()
bw_df = bw_df.with_columns(pl.col("query_name").alias("species"),
                           pl.col("match_name").alias("acc"))
bw_df = bw_df.select(["species", "acc", "containment", "intersect_hashes"])

# mimic the Web site search threshold of 0.1
bw_df = bw_df.filter(pl.col("containment") >= .05)
bw_df

species,acc,containment,intersect_hashes
str,str,f64,i64
"""s__Bariatricus sp004560705""","""SRR9866666""",0.689906,15002
"""s__Bariatricus sp004560705""","""SRR8960383""",0.681168,14812
"""s__Bariatricus sp004560705""","""SRR17241623""",0.677121,14724
"""s__Bariatricus sp004560705""","""ERR2241669""",0.677029,14722
"""s__Bariatricus sp004560705""","""SRR11185229""",0.676707,14715
…,…,…,…
"""s__UBA2868 sp004552595""","""ERR3801476""",0.05,1131
"""s__UBA2868 sp004552595""","""SRR19053357""",0.05,1131
"""s__UBA2868 sp004552595""","""SRR5558108""",0.05,1131


In [4]:
metadata_df = pl.scan_parquet('/group/ctbrowngrp5/sra-metagenomes/20241128-metadata.parquet')
metadata_df = metadata_df.select(["acc", "organism", "assay_type"])\
    .filter(pl.col("acc") != "NP")\
    .filter(pl.col("assay_type") == "WGS")

metadata_df = metadata_df.collect()
metadata_df

acc,organism,assay_type
str,str,str
"""SRR28523869""","""human metagenome""","""WGS"""
"""SRR19901137""","""Streptococcus suis""","""WGS"""
"""SRR24962475""","""Escherichia coli""","""WGS"""
"""SRR29161330""","""biofilm metagenome""","""WGS"""
"""SRR26668739""","""bovine gut metagenome""","""WGS"""
…,…,…
"""ERR4174288""","""human gut metagenome""","""WGS"""
"""ERR3160109""","""human gut metagenome""","""WGS"""
"""ERR2709457""","""human gut metagenome""","""WGS"""


In [5]:
bw_df = bw_df.join(metadata_df, on="acc", how="left")
bw_df

species,acc,containment,intersect_hashes,organism,assay_type
str,str,f64,i64,str,str
"""s__Bariatricus sp004560705""","""SRR9866666""",0.689906,15002,"""Lawsonia intracellularis""","""WGS"""
"""s__Bariatricus sp004560705""","""SRR8960383""",0.681168,14812,"""pig gut metagenome""","""WGS"""
"""s__Bariatricus sp004560705""","""SRR17241623""",0.677121,14724,"""pig gut metagenome""","""WGS"""
"""s__Bariatricus sp004560705""","""ERR2241669""",0.677029,14722,"""metagenome""","""WGS"""
"""s__Bariatricus sp004560705""","""SRR11185229""",0.676707,14715,"""pig gut metagenome""","""WGS"""
…,…,…,…,…,…
"""s__UBA2868 sp004552595""","""ERR3801476""",0.05,1131,"""bovine gut metagenome""","""WGS"""
"""s__UBA2868 sp004552595""","""SRR19053357""",0.05,1131,"""bovine gut metagenome""","""WGS"""
"""s__UBA2868 sp004552595""","""SRR5558108""",0.05,1131,"""human gut metagenome""","""WGS"""


In [6]:
bw_df['organism'].value_counts().sort(by='count', descending=True)

organism,count
str,u32
"""human gut metagenome""",701542
"""gut metagenome""",135908
null,130347
"""metagenome""",121881
"""pig gut metagenome""",80607
…,…
"""Procyon lotor""",1
"""Acrocephalus sechellensis""",1
"""Klebsiella oxytoca""",1


In [7]:
rename_df = []
for organism in bw_df['organism'].unique().to_list():
    simpleorg = 'unknown'
    if organism:
        organism = organism.lower()
        for kw in ['human', 'homo']:
            if kw in organism.lower():
                simpleorg = 'human'
                break
        for kw in ['pig', 'sus', 'scrofa']:
            if kw in organism.lower():
                simpleorg = 'pig'
                break
    rename_df.append(dict(organism=organism, simpleorg=simpleorg))

rename_df = pl.DataFrame(rename_df)
rename_df['simpleorg'].value_counts().sort(by='count', descending=True)


simpleorg,count
str,u32
"""unknown""",347
"""human""",17
"""pig""",7


In [8]:
rename_df

organism,simpleorg
str,str
"""air metagenome""","""unknown"""
"""[ruminococcus] torques""","""unknown"""
"""biocathode metagenome""","""unknown"""
"""drinking water metagenome""","""unknown"""
"""subdoligranulum sp. ps108""","""unknown"""
…,…
"""coprococcus""","""unknown"""
"""clostridium paraputrificum""","""unknown"""
"""dama dama""","""unknown"""


In [9]:
# add a new column with a simplified organism
bw_df = bw_df.join(rename_df, on='organism', how='inner', coalesce=True).select(['acc', 'organism', 'species', 'simpleorg'])
bw_df.select(['acc', 'simpleorg']).unique()['simpleorg'].value_counts()


simpleorg,count
str,u32
"""unknown""",86413
"""pig""",6602
"""human""",151690


In [10]:
bw_df

acc,organism,species,simpleorg
str,str,str,str
"""SRR8960383""","""pig gut metagenome""","""s__Bariatricus sp004560705""","""pig"""
"""SRR17241623""","""pig gut metagenome""","""s__Bariatricus sp004560705""","""pig"""
"""ERR2241669""","""metagenome""","""s__Bariatricus sp004560705""","""unknown"""
"""SRR11185229""","""pig gut metagenome""","""s__Bariatricus sp004560705""","""pig"""
"""ERR2241674""","""metagenome""","""s__Bariatricus sp004560705""","""unknown"""
…,…,…,…
"""SRR12529321""","""gut metagenome""","""s__UBA2868 sp004552595""","""unknown"""
"""ERR3801476""","""bovine gut metagenome""","""s__UBA2868 sp004552595""","""unknown"""
"""SRR19053357""","""bovine gut metagenome""","""s__UBA2868 sp004552595""","""unknown"""


In [11]:
def make_matrices(df):
    acc_df = df['acc'].unique().to_frame().with_row_index(name='acc_index')
    species_df = df['species'].unique().to_frame().with_row_index(name='species_index')
    org_df = df['simpleorg'].unique().to_frame().with_row_index(name='org_index')

    df = df.join(org_df, on='simpleorg', how='left')
    df = df.join(acc_df, on='acc', how='left')
    df = df.join(species_df, on='species', how='left')

    obs = np.zeros((len(acc_df), len(species_df)))
    target = np.zeros((len(acc_df)))

    for row in df.iter_rows(named=True):
        acc_id = row["acc_index"]
        org_id = row["org_index"]
        species_id = row["species_index"]

        obs[acc_id, species_id] = 1
        target[acc_id] = org_id

    print(f'observations matrix shape is: {obs.shape}')

    return df, obs, target


In [12]:
human_pig_only = bw_df.filter(pl.col("simpleorg") != "unknown")
hp_df, hp_obs, hp_target = make_matrices(human_pig_only)

observations matrix shape is: (158292, 16)


In [13]:
hp_df

acc,organism,species,simpleorg,org_index,acc_index,species_index
str,str,str,str,u32,u32,u32
"""SRR8960383""","""pig gut metagenome""","""s__Bariatricus sp004560705""","""pig""",1,17160,14
"""SRR17241623""","""pig gut metagenome""","""s__Bariatricus sp004560705""","""pig""",1,133616,14
"""SRR11185229""","""pig gut metagenome""","""s__Bariatricus sp004560705""","""pig""",1,116194,14
"""SRR17241692""","""pig gut metagenome""","""s__Bariatricus sp004560705""","""pig""",1,111350,14
"""SRR17241693""","""pig gut metagenome""","""s__Bariatricus sp004560705""","""pig""",1,21808,14
…,…,…,…,…,…,…
"""SRR11183595""","""pig gut metagenome""","""s__UBA2868 sp004552595""","""pig""",1,134311,9
"""SRR7825890""","""human gut metagenome""","""s__UBA2868 sp004552595""","""human""",0,45921,9
"""ERR11457552""","""human gut metagenome""","""s__UBA2868 sp004552595""","""human""",0,48724,9


## 6-Fold Cross Validation approach

Run 6 different splits of the data; train on 1/6 of the data and test on the other 5/6s of the data, and then repeat that for each of the 6 splits.

In [14]:
kf = StratifiedKFold(n_splits=6)

accuracies = []
i = 1
for train_sub, test_sub in kf.split(hp_obs, hp_target):
    dt = DecisionTreeClassifier(random_state=42)
    tree = dt.fit(hp_obs[train_sub], hp_target[train_sub])
    pred = tree.predict(hp_obs[test_sub])

    accuracy = balanced_accuracy_score(hp_target[test_sub], pred)
    accuracies.append(accuracy)
    print(f"iteration {i}: accuracy {accuracy:.3f}")
    i += 1

print(f"mean accuracy across {i-1} splits: {np.mean(accuracies):.3f}")

iteration 1: accuracy 0.977
iteration 2: accuracy 0.986
iteration 3: accuracy 0.985
iteration 4: accuracy 0.981
iteration 5: accuracy 0.980
iteration 6: accuracy 0.981
mean accuracy across 6 splits: 0.981


## What do we after 6-fold cross validation?

This approach lets us check to see that, in general, the performance of any classifer trained on this data will be pretty good - independent of which set of data we use.

Now, to train the best possible classifier for _future_ use, we will use all the data to train:

In [15]:
dt = DecisionTreeClassifier(random_state=42)
tree = dt.fit(hp_obs, hp_target)
pred = tree.predict(hp_obs)

accuracy = balanced_accuracy_score(hp_target, pred)
print(f"full classifier: accuracy {accuracy:.3f}")

full classifier: accuracy 0.984


## What do we do with the full classifier now??

Now we can go back to the original data - not just the human/pig subset, but the one with unknowns. (Or, potentially, new data sets.)

The only thing we need to be careful about is to encode new observations using the species indexes that match the trained classifier, as well as the organism indexes (0 pig, 1 human). Let's extract those:

In [16]:
species_index_df = hp_df.select(['species_index', 'species']).unique()
species_index_df

species_index,species
u32,str
9,"""s__UBA2868 sp004552595"""
15,"""s__Gemmiger qucibialis"""
13,"""s__Prevotella sp000434975"""
6,"""s__Holdemanella porci"""
12,"""s__Roseburia inulinivorans"""
…,…
2,"""s__Sodaliphilus sp004557565"""
14,"""s__Bariatricus sp004560705"""
7,"""s__Phocaeicola vulgatus"""


In [17]:
org_df = hp_df.select(['org_index', 'simpleorg']).unique()
org_df

org_index,simpleorg
u32,str
1,"""pig"""
0,"""human"""


Now, let's take some of some of the data we didn't touch from the original bw_df, and encode and classify it: 

In [18]:
def make_obs_matrix(df, species_df):
    # need a new access mapping
    acc_df = df['acc'].unique().to_frame().with_row_index(name='acc_index')

    # construct new internal data frame, populate
    df = df.join(acc_df, on='acc', how='left')
    df = df.join(species_df, on='species', how='left')

    obs = np.zeros((len(acc_df), len(species_df)))

    for row in df.iter_rows(named=True):
        acc_id = row["acc_index"]
        species_id = row["species_index"]

        obs[acc_id, species_id] = 1

    print(f'observations matrix shape is: {obs.shape}')

    return df, obs


In [19]:
unknown_only_df = bw_df.filter(pl.col("simpleorg") == "unknown")
unk_df, unk_obs = make_obs_matrix(unknown_only_df, species_index_df)

observations matrix shape is: (86413, 16)


In [20]:
unknown_pred = tree.predict(unk_obs)
unknown_pred

array([0., 1., 0., ..., 1., 0., 0.], shape=(86413,))

## Predictions are great and all but...

Great! XXX predictions 😭 . What do we do with them??

Let's merge the predictions back into the spreadsheet.

In [21]:
unk_pred_df = []
for (row_n, predicted_org) in enumerate(unknown_pred):
    unk_pred_df.append(dict(acc_index=row_n, org_index=int(predicted_org)))
unk_pred_df = pl.DataFrame(unk_pred_df).join(org_df, on='org_index', how='left')

unk_acc_df = unk_df.select(["acc", "acc_index"]).unique()

unk_pred_df = unk_pred_df.join(unk_acc_df, on='acc_index', how='left')
unk_pred_df

acc_index,org_index,simpleorg,acc
i64,i64,str,str
0,0,"""human""","""SRR23946406"""
1,1,"""pig""","""SRR7060080"""
2,0,"""human""","""SRR5829310"""
3,0,"""human""","""ERR2749814"""
4,0,"""human""","""SRR22877617"""
…,…,…,…
86408,0,"""human""","""SRR2193947"""
86409,0,"""human""","""SRR6474237"""
86410,1,"""pig""","""SRR17543904"""


In [22]:
unk_pred_df["simpleorg"].value_counts().sort(by='count', descending=True)

simpleorg,count
str,u32
"""human""",79012
"""pig""",7401


## OK! We have our predictions... did this actually work?

In [23]:
# sample 5 randomly
unk_pred_df.sample(n=5)

acc_index,org_index,simpleorg,acc
i64,i64,str,str
42908,0,"""human""","""SRR20950640"""
55488,0,"""human""","""SRR19843651"""
52739,0,"""human""","""SRR29291470"""
46353,0,"""human""","""ERR1989794"""
2218,0,"""human""","""SRR6713632"""


In [24]:
# sample 5 predicted as pig
unk_pred_df.filter(pl.col("simpleorg") == "pig").sample(n=5)

acc_index,org_index,simpleorg,acc
i64,i64,str,str
77619,1,"""pig""","""DRR507840"""
64471,1,"""pig""","""ERR1915705"""
76639,1,"""pig""","""ERR6786098"""
28798,1,"""pig""","""ERR1915717"""
14825,1,"""pig""","""SRR20950976"""


In [25]:
# sample 5 predicted as human
unk_pred_df.filter(pl.col("simpleorg") == "human").sample(n=5)

acc_index,org_index,simpleorg,acc
i64,i64,str,str
8686,0,"""human""","""ERR2227802"""
42710,0,"""human""","""SRR13236272"""
61096,0,"""human""","""SRR11599097"""
12381,0,"""human""","""SRR12179783"""
22509,0,"""human""","""SRR26191730"""


# What features are most important for good classification?

First, let's get the feature importances.

In [26]:
importances_df = []
for (species_index, importance) in enumerate(tree.feature_importances_):
    d = dict(species_index=species_index, importance=importance)
    importances_df.append(d)

importances_df = pl.DataFrame(importances_df)
importances_df = importances_df.join(species_index_df, on='species_index', how='left')
importances_df = importances_df.sort(by='importance', descending=True)
importances_df

species_index,importance,species
i64,f64,str
8,0.822327,"""s__Lactobacillus amylovorus"""
2,0.06974,"""s__Sodaliphilus sp004557565"""
12,0.048809,"""s__Roseburia inulinivorans"""
1,0.011277,"""s__JAFBIX01 sp021531895"""
15,0.008583,"""s__Gemmiger qucibialis"""
…,…,…
3,0.001859,"""s__Prevotella sp002251295"""
10,0.001781,"""s__Fimisoma sp002320005"""
0,0.001171,"""s__Cryptobacteroides sp9005469…"


Now let's rebuild the classifier after removing each species, and see how well it does...

In [27]:
species_in_order = importances_df['species'].to_list()
species_in_order

['s__Lactobacillus amylovorus',
 's__Sodaliphilus sp004557565',
 's__Roseburia inulinivorans',
 's__JAFBIX01 sp021531895',
 's__Gemmiger qucibialis',
 's__Prevotella sp000434975',
 's__UBA2868 sp004552595',
 's__JALFVM01 sp022787145',
 's__Phocaeicola vulgatus',
 's__Holdemanella porci',
 's__Mogibacterium_A kristiansenii',
 's__Prevotella sp002251295',
 's__Fimisoma sp002320005',
 's__Cryptobacteroides sp900546925',
 's__Phascolarctobacterium_A succinatutens',
 's__Bariatricus sp004560705']

In [28]:
human_pig_only = bw_df.filter(pl.col("simpleorg") != "unknown")
species_to_rm = list(species_in_order)

remove_df = human_pig_only
while len(species_to_rm) >= 2:
    rm_species = species_to_rm.pop(0)
    print(f'removing {rm_species} too')

    remove_df = remove_df.filter(pl.col("species") != rm_species)
    augmented_df, rm_obs, rm_target = make_matrices(remove_df)

    dt = DecisionTreeClassifier(random_state=42)
    tree = dt.fit(rm_obs, rm_target)
    pred = tree.predict(rm_obs)

    accuracy = balanced_accuracy_score(rm_target, pred)
    print(f"classifier accuracy with {remove_df['species'].n_unique()} is: {accuracy:.3f}")
    print('---')

removing s__Lactobacillus amylovorus too
observations matrix shape is: (158105, 15)
classifier accuracy with 15 is: 0.988
---
removing s__Sodaliphilus sp004557565 too
observations matrix shape is: (158080, 14)
classifier accuracy with 14 is: 0.948
---
removing s__Roseburia inulinivorans too
observations matrix shape is: (155234, 13)
classifier accuracy with 13 is: 0.941
---
removing s__JAFBIX01 sp021531895 too
observations matrix shape is: (155048, 12)
classifier accuracy with 12 is: 0.940
---
removing s__Gemmiger qucibialis too
observations matrix shape is: (149771, 11)
classifier accuracy with 11 is: 0.963
---
removing s__Prevotella sp000434975 too
observations matrix shape is: (148769, 10)
classifier accuracy with 10 is: 0.955
---
removing s__UBA2868 sp004552595 too
observations matrix shape is: (148757, 9)
classifier accuracy with 9 is: 0.896
---
removing s__JALFVM01 sp022787145 too
observations matrix shape is: (148598, 8)
classifier accuracy with 8 is: 0.894
---
removing s__Phoca